# Лекция 1: Примеры NLP на русском языке (spaCy)

В этом ноутбуке показаны 3 задачи:

1. Определение смысла слова (по контексту)
2. Анализ эмоционального окраса
3. Распознавание именованных сущностей (NER)

Во всех шагах используются русские примеры и короткая проверка результата.

In [ ]:
# Установка библиотек (выполнить один раз)
!pip install -q spacy
!python -m spacy download ru_core_news_md

In [ ]:
# !uv run python -m spacy download ru_core_news_md # TODO: Only for uv users

In [ ]:
# https://huggingface.co/spacy/ru_core_news_md

In [ ]:
# ru_core_news_sm — меньше и быстрее, но слабее
# ru_core_news_lg — больше и точнее, но тяжелее
# ru_core_news_md - баланс точности и скорости

In [3]:
import spacy

nlp = spacy.load("ru_core_news_md")
print("Модель загружена:", nlp.meta.get("name"))

Модель загружена: core_news_md


# Задача: Определение смысла слова
## Как работает (по какому принципу, словари и т.п.)
Один из простых подходов: сравнивать контекст предложения с «эталонными» описаниями смыслов.
Вместо ручных словарей можно использовать векторные представления текста (эмбеддинги spaCy) и выбирать смысл с максимальным сходством.

In [4]:
## Пример кода, который это делает
target_word = "лук"

sense_examples = {
    "оружие": "Лучник натянул тетиву и выстрелил из лука в цель.",
    "овощ": "Для салата нужен свежий зеленый лук и укроп."
}

sentence = "Я добавил лук в суп."
context_doc = nlp(sentence)

scores = {}
for sense, example in sense_examples.items():
    scores[sense] = context_doc.similarity(nlp(example))

predicted_sense = max(scores, key=scores.get)

print("Предложение:", sentence)
print("Оценки похожести:", scores)
print("Предсказанный смысл слова '", target_word, "': ", predicted_sense, sep="")

Предложение: Я добавил лук в суп.
Оценки похожести: {'оружие': 0.5074542145680851, 'овощ': 0.571432042899425}
Предсказанный смысл слова 'лук': овощ


## Пояснение к выполнению
- Мы описали два значения слова `лук` через примеры.
- Считаем сходство контекста предложения с каждым примером.
- Значение с наибольшим сходством принимаем как наиболее вероятное.

## Как работает `similarity` в spaCy

`Doc.similarity(other_doc)` сравнивает векторные представления двух текстов.
- у каждого слова есть вектор (числовое представление смысла),
- для предложения берется усредненный вектор,
- затем считается косинусная близость между векторами (значение от -1 до 1) (поговорим о расстояниях позже)

Чем выше число, тем ближе тексты по смыслу.

### Простенький подход (без сложных моделей)
Можно сравнивать не векторы, а пересечение лемм:
- нормализуем слова через spaCy (`lemma_`),
- убираем стоп-слова,
- считаем долю общих лемм.

Такой метод проще, но хуже понимает синонимы и скрытый контекст.

In [11]:
## Примеры для сравнения

def lemma_set(text: str):
    doc = nlp(text)
    return {t.lemma_.lower() for t in doc if t.is_alpha and not t.is_stop}

def overlap_score(a: str, b: str) -> float:
    a_set, b_set = lemma_set(a), lemma_set(b)

    # a_set & b_set: пересечение (общие леммы)
    # a_set | b_set: объединение (все уникальные леммы из обоих текстов)
    
    return len(a_set & b_set) / len(a_set | b_set)

examples = [
    # 1) Лексическое совпадение
    ("Я купил зеленый лук", "В салат добавили лук и укроп"),
    # 2) Контекст предметной области
    ("Лук нарезали для супа", "Овощи отправили в кастрюлю"),
    # 3) Другой смысл слова
    ("Лучник натянул лук", "Лучник выстрелил стрелой в цель"),
    # 4) Слабое тематическое совпадение
    ("В магазине купили овощи", "На стадионе начался матч"),
    # 5) Почти одинаковый смысл разными словами
    ("Мне понравился этот фильм", "Картина произвела на меня хорошее впечатление"),
]

for i, (a, b) in enumerate(examples, 1):
    vec_sim = nlp(a).similarity(nlp(b))
    lex_sim = overlap_score(a, b)
    print(f"Пример {i}")
    print("A:", a)
    print("B:", b)
    print(f"  spaCy similarity: {vec_sim:.3f}")
    print(f"  overlap лемм:     {lex_sim:.3f}")
    print("-" * 50)

print("Принцип сравнения:")
print("- similarity лучше ловит общий смысл;")
print("- overlap лемм лучше показывает буквальные совпадения слов.")

Пример 1
A: Я купил зеленый лук
B: В салат добавили лук и укроп
  spaCy similarity: 0.538
  overlap лемм:     0.167
--------------------------------------------------
Пример 2
A: Лук нарезали для супа
B: Овощи отправили в кастрюлю
  spaCy similarity: 0.212
  overlap лемм:     0.000
--------------------------------------------------
Пример 3
A: Лучник натянул лук
B: Лучник выстрелил стрелой в цель
  spaCy similarity: 0.147
  overlap лемм:     0.167
--------------------------------------------------
Пример 4
A: В магазине купили овощи
B: На стадионе начался матч
  spaCy similarity: 0.022
  overlap лемм:     0.000
--------------------------------------------------
Пример 5
A: Мне понравился этот фильм
B: Картина произвела на меня хорошее впечатление
  spaCy similarity: 0.525
  overlap лемм:     0.000
--------------------------------------------------
Принцип сравнения:
- similarity лучше ловит общий смысл;
- overlap лемм лучше показывает буквальные совпадения слов.


# Задача: Анализ эмоционального окраса
## Как работает (по какому принципу, словари и т.п.)
Базовый подход: словарь положительных и отрицательных слов.
spaCy помогает нормализовать слова (лемматизация), чтобы сравнение шло по начальной форме.

In [5]:
## Пример кода, который это делает
positive_lemmas = {"отличный", "радость", "счастливый", "прекрасный", "любить"}
negative_lemmas = {"плохой", "грусть", "ужасный", "ненавидеть", "печальный"}

text = "Мне очень понравился фильм, он был прекрасный и интересный."
doc = nlp(text)
lemmas = [t.lemma_.lower() for t in doc if t.is_alpha]

pos_count = sum(1 for l in lemmas if l in positive_lemmas)
neg_count = sum(1 for l in lemmas if l in negative_lemmas)

if pos_count > neg_count:
    sentiment = "позитивная"
elif neg_count > pos_count:
    sentiment = "негативная"
else:
    sentiment = "нейтральная"

print("Текст:", text)
print("Леммы:", lemmas)
print("Позитивных:", pos_count, "| Негативных:", neg_count)
print("Эмоциональная окраска:", sentiment)

Текст: Мне очень понравился фильм, он был прекрасный и интересный.
Леммы: ['мне', 'очень', 'понравиться', 'фильм', 'он', 'быть', 'прекрасный', 'и', 'интересный']
Позитивных: 1 | Негативных: 0
Эмоциональная окраска: позитивная


## Пояснение к выполнению
- Сначала получаем леммы русского текста.
- Затем считаем совпадения с позитивным и негативным словарями.
- По балансу совпадений определяем итоговую эмоциональную окраску.
- Для более высокой точности обычно используют обученные модели (например, ruBERT).

# Задача: Распознавание именованных сущностей
## Как работает (по какому принципу, словари и т.п.)
NER-модель spaCy определяет в тексте сущности: людей, организации, локации и т.д.
Модель обучена на размеченных корпусах, где сущности выделены заранее.

In [ ]:
# https://data-light.ru/blog/named-entity-recognition/#:~:text=NER%20(Named%20Entity%20Recognition%2C%20распознавание,Эти%20категории%20называются%20именованными%20сущностями.


In [6]:
## Пример кода, который это делает
text = "Иван Петров работает в компании Яндекс в Москве."
doc = nlp(text)

print("Текст:", text)
print("Найденные сущности:")
for ent in doc.ents:
    print(f"- {ent.text} -> {ent.label_}")

Текст: Иван Петров работает в компании Яндекс в Москве.
Найденные сущности:
- Иван Петров -> PER
- Яндекс -> ORG
- Москве -> LOC


## Пояснение к выполнению
- Передаем русский текст в модель `ru_core_news_md`.
- `doc.ents` возвращает найденные сущности.
- Для проверки шага достаточно убедиться, что модель выделила имя, организацию и географическое место.

## Проверка шагов
Если при запуске возникают ошибки:
1. Проверьте, что выполнились команды установки `!pip install ...`.
2. Проверьте загрузку модели: `python -m spacy download ru_core_news_md`.
3. Запускайте ячейки сверху вниз по порядку.

После этого все примеры должны работать на русском языке.